<a href="https://colab.research.google.com/github/cindykhris/SummerInternship2020/blob/master/SummerInternship2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pine Biotech Summer Reasearch

**Cindy Pino**


Abstract: In this study, we will analyze the gene expression differnt coronaviruses (CoVs) infections to determine how each virus differs when causing a disease. Here, we focuses the analysis on samples from SARS-CoV-1, MERS, and SARS-CoV-2. 
The raw sequence data (fastq files) for the SARS-CoV-1 and MERS infections were downloaded from GEO [GSE56192](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE56192), including their corresponding Mock-treated controls). The raw sequence data (fastq files) for the SARS-CoV-2 infections were downloaded from GEO ([GSE147507](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE147507)), including their corresponding Mock-treated controls). 

In this Jupyter notebook, I will use both R and Python to analyze three viruses: MERS, SARS-CoV-1, amd SARS-CoV-2

##Using R and Python in the same Notebook
First, let's active R magic. Don't forget to use %%R before running a R code 


---

In [ ]:
#active R magic
%load_ext rpy2.ipython

##Import all the folders we will need for this files (R and Python)

###Python

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


###R

In [ ]:
%%R
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install("DESeq2")

R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/BiocManager_1.30.10.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 40205 bytes (39 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[

In [ ]:
%%R
library(DESeq2)


R[write to console]: Loading required package: S4Vectors

R[write to console]: Loading required package: stats4

R[write to console]: Loading required package: BiocGenerics

R[write to console]: Loading required package: parallel

R[write to console]: 
Attaching package: ‘BiocGenerics’


R[write to console]: The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


R[write to console]: The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


R[write to console]: The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, 

In [ ]:
%%R
library(tidyverse)

R[write to console]: ── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

R[write to console]: ✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.1     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

R[write to console]: ── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::collapse()   masks IRanges::collapse()
✖ dplyr::combine()    masks Biobase::combine(), BiocGenerics::combine()
✖ dplyr::count()      masks matrixStats::count()
✖ dplyr::desc()       masks IRanges::desc()
✖ tidyr::expand()     masks S4Vectors::expand()
✖ dplyr::filter()     masks stats::filter()
✖ dplyr::first()      masks S4Vectors::first()
✖ dplyr::lag()        masks stats::lag()
✖ ggplot2::Position() masks BiocGenerics::Position(), base::Position()
✖ purrr::reduce()     masks GenomicRanges::reduce(), IRanges::reduce()
✖ dplyr::rename()     masks S4Vectors::rename()
✖ purrr::simplify()   masks DelayedArray::simp

In [ ]:
%%R
install.packages('dplyr', lib = "/usr/lib/R/library")

R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/dplyr_1.0.2.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 935654 bytes (913 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]:

In [ ]:
%%R
install.packages('EnhancedVolcano',lib = "/usr/lib/R/library")
library(EnhancedVolcano)

R[write to console]: Error in library(EnhancedVolcano) : 
  there is no package called ‘EnhancedVolcano’
Calls: <Anonymous> ... <Anonymous> -> <Anonymous> -> withVisible -> library

R[write to console]: In addition: 
R[write to console]: Warning messages:

R[write to console]: 1: 
R[write to console]: In file.create(f.tg) :
R[write to console]: 
 
R[write to console]:  cannot create file '/usr/lib/R/doc/html/packages.html', reason 'No such file or directory'

R[write to console]: 2: 
R[write to console]: In make.packages.html(.Library) :
R[write to console]:  cannot update HTML package index

R[write to console]: 3: package ‘EnhancedVolcano’ is not available (for R version 3.6.3) 




Error in library(EnhancedVolcano) : 
  there is no package called ‘EnhancedVolcano’
Calls: <Anonymous> ... <Anonymous> -> <Anonymous> -> withVisible -> library


In [ ]:
%%R
install.packages('ggplot',lib = "/usr/lib/R/library")



## SARS-CoV
Severe Acute Respiratory Syndrom Coronavirus (SARS-CoV) was first identified in 20013 (WHO). 

**Pipeline Workflow**



*   PreProcessing
    
      * PCR clean - to remove duplicates from the PCR run  thus reducing redundancy. 
      * Trimmomatic - removes adapter sequences
*   Mapping 
      * Bowtie-2t
* Quantification 

    * RSEM -  Fragment Per Kilobase of transcript per Million mapped reads  for Paired-End Reads







### Upload and clean the file 
The dataset contains the ene expression of SARS and Mocks samples. 

* log transformation and noralization
* filter out genes with very low expression (indistinguishable from noise
* RNA-seq visualization 
    * histagram, boxplot, heatmap 

It is not practical to try and visualize all genes, we have to reduce the complexity (or dimensionality) of this data. 

In [ ]:
%%R
df = read.table('drive/My Drive/SummerResearch/DESeq_SARS_expression_genes_FPKM.txt',skip = 1, header = TRUE)
head(df,2)

  ENSG00000000003.13 X14.00 X14.17 X14.05 X13.24 X15.31 X14.33 X12.93 X13.14
1 ENSG00000000419.11  42.43  41.34  51.64  50.84  42.00  42.58  59.62  59.73
2 ENSG00000000457.12   3.07   2.84   2.63   2.63   2.88   3.25   2.65   2.85
  X13.07 X13.86 X12.87 X12.67
1  51.21  50.55  56.24  55.60
2   2.65   2.95   2.65   2.59


In [ ]:
%%R

datanew <- filter(df, variable ==0)

R[write to console]: Error: Problem with `filter()` input `..1`.
✖ object 'variable' not found
ℹ Input `..1` is `variable == 0`.
Backtrace:
     █
  1. ├─(new("nonstandardGenericFunction", .Data = function (expr, envir = parent.frame(), ...
  2. ├─(new("nonstandardGenericFunction", .Data = function (expr, envir = parent.frame(), ...
  3. │ └─(new("nonstandardGenericFunction", .Data = function (expr, envir = parent.frame(), ...
  4. ├─base::withVisible(...)
  5. ├─dplyr::filter(df, variable == 0)
  6. ├─dplyr:::filter.data.frame(df, variable == 0)
  7. │ └─dplyr:::filter_rows(.data, ...)
  8. │   ├─base::withCallingHandlers(...)
  9. │   └─mask$eval_all_filter(dots, env_filter)
 10. └─base::.handleSimpleError(...)
 11.   └─dplyr:::h(simpleError(msg, call))




Error: Problem with `filter()` input `..1`.
✖ object 'variable' not found
ℹ Input `..1` is `variable == 0`.
Backtrace:
     █
  1. ├─(new("nonstandardGenericFunction", .Data = function (expr, envir = parent.frame(), ...
  2. ├─(new("nonstandardGenericFunction", .Data = function (expr, envir = parent.frame(), ...
  3. │ └─(new("nonstandardGenericFunction", .Data = function (expr, envir = parent.frame(), ...
  4. ├─base::withVisible(...)
  5. ├─dplyr::filter(df, variable == 0)
  6. ├─dplyr:::filter.data.frame(df, variable == 0)
  7. │ └─dplyr:::filter_rows(.data, ...)
  8. │   ├─base::withCallingHandlers(...)
  9. │   └─mask$eval_all_filter(dots, env_filter)
 10. └─base::.handleSimpleError(...)
 11.   └─dplyr:::h(simpleError(msg, call))


In [ ]:
%%R
# Cleaning the file for processing
ColNames1 <- df$id #take the column name
df = df[,-1] #take the data part - numeric values
df=as.matrix(df)


R[write to console]: Error in df$id : $ operator is invalid for atomic vectors
Calls: <Anonymous> -> <Anonymous> -> <Anonymous> -> withVisible




Error in df$id : $ operator is invalid for atomic vectors
Calls: <Anonymous> -> <Anonymous> -> <Anonymous> -> withVisible


In [ ]:
%%R
names(df) <- NULL


In [ ]:
%%R
#Basic Settings:

colors <- c(rep('red',6),rep('blue',3),rep('green',2),rep('gray',2))
par(mar=c(14,4,2,2))
boxplot(df, main="Gene Expression",las = 2, cex.axis=0.6)


In [ ]:

%%R
#Log transformation
logdata <- log(df+1)
par(mar=c(14,4,2,2))
boxplot(logdata, main="Log Transformed Gene Expression", col = 'blue', las = 2, cex.axis=0.6)


In [ ]:
%%R
#Descriptive statistics
summary(df)

In [ ]:
%%R
hist(df, col='pink')


In [ ]:
%%R
hist(logdata, col='magenta')

In [ ]:
%%R
barplot(sort(logdata[100,]), col = "blue", main = ColNames1[2], font.axis=1, cex.axis=1, las=2)


In [ ]:
%%R
heatmap(logdata)